In [1]:
import os
import json
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# from kobert_tokenizer import KoBERTTokenizer
from transformers import AutoModel, AutoTokenizer


folders = ["Clickbait_Auto", "Clickbait_Direct", "NonClickbait_Auto"]
categories = ["EC", "ET", "GB", "IS", "LC", "PO", "SO"]
categories = ["EC"]

atricles_per_category = 10

def extract_part1(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        news_title = data['labeledDataInfo']['newTitle']
        news_content = data['sourceDataInfo']['newsContent']
        return news_title, news_content
    
def extract_part2(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        news_title = data['sourceDataInfo']['newsTitle']
        news_content = data['labeledDataInfo']['processSentenceInfo']
        content = ""
        for sentence in news_content:
            content += sentence["sentenceContent"] + " "
        return news_title, content


news_titles = []
news_contents = []
labels = []

folder_path = './data/Part1'

for folder in folders:
    for category in categories:
        path = folder_path+"/"+folder+"/"+category
        count = 0
        for filename in os.listdir(path):
            count += 1
            if count >= (atricles_per_category *2 if folder == "NonClickbait_Auto" else atricles_per_category):
                break
            file_path = os.path.join(path, filename)
            if os.path.isfile(file_path):
                try:
                    title, content = extract_part1(file_path)
                    news_titles.append(title)
                    news_contents.append(content)
                    labels.append([0, 1] if folder == "NonClickbait_Auto" else [1, 0])
                except (json.JSONDecodeError, KeyError) as e:
                    print(f"Error processing {filename}: {e}")
                    continue

# folder_path = './data/Part2'

# for folder in folders:
#     for category in categories:
#         path = folder_path+"/"+folder+"/"+category
#         count = 0
#         for filename in os.listdir(path):
#             count+=1
#             if count >= atricles_per_category:
#                 break
#             file_path = os.path.join(path, filename)
#             if os.path.isfile(file_path):
#                 try:
#                     title, content = extract_part2(file_path)
#                     news_titles.append(title)
#                     news_contents.append(content)
#                     labels.append([0,1]if folder == "NonClickbait_Auto" else [1,0])
#                 except (json.JSONDecodeError, KeyError) as e:
#                     print(f"Error processing {filename}: {e}")
#                     continue


print("title:", news_titles[:5])
print(len(news_titles))

title: ["삼성준법위, 노사갈등·ESG '실질적 변화'에 힘 싣는다", "주택아파트 마련, 담보대출금리비교 활용해서 '알뜰하게'", '정은보 금감원장 \\"우리銀 횡령 엄정 조치…내부통제 제도 개선도\\"', '대전충남지역 2월 자금사정 밝다', "쿠팡 '전 상품 품절' 오류 발생"]
37


In [2]:
data = []

for i in range(len(labels)):
    data.append([news_titles[i], news_contents[i], labels[i]])

del news_titles
del news_contents
del labels

random.shuffle(data)

train_titles = []
train_contents = []
labels = []

for d in data:
    train_titles.append(d[0])
    train_contents.append(d[1])
    labels.append(d[2])

In [3]:
print(train_titles[16])
print(train_contents[16])
print(labels[16])

아시아나항공 자본 확충 추진...국제선 시동거는 대형항공사
과학기술계 정부출연연구기관의 '비정규직 정규직 전환'이 내년까지 본격적으로 추진된다.
전환대상자는 상시·지속 업무를 수행하는 현재 비정규직 근무자로, 필요하다면 '경쟁채용' 방식이 도입된다.
과학기술정보통신부는 이러한 내용을 담은 '출연연 비정규직의 정규직 전환 가이드라인'을 24일 발표했다.
이는 지난 7월 관계부처 합동으로 발표한 '공공부문 정규직 전환 가이드라인'의 후속조치다.
출연연은 기관별 임무에 따라 R&D(연구개발)를 수행하면서 전문 연구인력, 연구보조 인력 등을 비정규직 형태로 운영해왔다.
출연연 내 비정규직은 운영 방식과 비정규직 근무자가 수행하는 업무 특성이 다양해 일률적인 기준을 적용하기엔 무리가 있다.
과기정통부는 정부 정책 취지인 '상시·지속업무는 정규직으로 전환한다'는 원칙은 지키되 상시·지속업무의 범위를 확대했다.
그 예로 기간이 한정된 연구 프로젝트 수행을 위해 채용한 비정규직도 통상적으로 계약을 연장하며 다년 간 또는 다수 프로젝트를 수행하기 때문에 상시·지속 업무로 간주할 수 있다.
또 연구 수행 시 안전과 관련이 깊거나 폭발물·유해물질 처리 등 위험이 큰 업무는 정규직화 한다.
우선 전환대상 업무가 결정되면 해당 업무를 수행 중인 현 근무자를 대상으로 최소한의 평가절차를 거쳐 정규직으로 전환한다.
평가과정에서 연구업무의 전문성 등 합리적인 사유가 있다면, 경쟁채용 방식이 적용될 수도 있다.
그러나 '정규직 전환 심의위원회'에서 기관이 제시한 사유와 현 근무자의 의견 등을 심의해 정당성을 확보돼야만 한다.
이날 발표된 가이드라인에 따라, 출연연은 12월까지 정규직 전환 계획을 확정해야 한다.
계획에 따라 출연연은 기간제의 경우 내년 3월까지 전환을 완료하고, 파견·용역직의 경우 내년 이후 민간업체의 계약 기간 종료 시점에 맞춰 전환해야 한다.
기관별 계획이 취합돼야 전체 대상자 수를 알 수 있을 전망이다.
반면 박사후연구원, 학생연구원 등 정규 직업을 갖기 전 '연수'가

In [5]:
cnt=0 # # of clickbait
for label in labels:
    if label == [1,0]:
        cnt+=1
print(cnt)

18


### Preprocessing

### Model

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from transformers import BertTokenizer, BertModel

class NewsConsistencyDataset(Dataset):
    def __init__(self, headlines, article_bodies, labels, tokenizer, max_headline_len=128, max_article_len=512):
        self.headlines = headlines
        self.article_bodies = article_bodies
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_headline_len = max_headline_len
        self.max_article_len = max_article_len

    def __len__(self):
        return len(self.headlines)
    def __getitem__(self, idx):
        headline = str(self.headlines[idx])
        article = str(self.article_bodies[idx])
        
        headline_encoding = self.tokenizer.encode_plus(
            headline,
            add_special_tokens=True,
            max_length=self.max_headline_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        article_encoding = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            max_length=self.max_article_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'headline_ids': headline_encoding['input_ids'].squeeze(0),
            'headline_mask': headline_encoding['attention_mask'].squeeze(0),
            'article_ids': article_encoding['input_ids'].squeeze(0),
            'article_mask': article_encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

class CrossAttention(nn.Module):
    def __init__(self, hidden_size, num_heads=8):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_size, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(hidden_size)
        self.linear = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value, key_padding_mask=None):
        attn_output, _ = self.attention(query, key, value, key_padding_mask=key_padding_mask)
        attn_output = self.norm(attn_output + query)
        return self.linear(attn_output)

class NewsConsistencyChecker(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
        self.bert = AutoModel.from_pretrained("monologg/kobert")

        for param in self.bert.parameters():
            param.requires_grad = False
        
        self.hidden_size = self.bert.config.hidden_size
        self.reduced_size = self.hidden_size // 2
        
        self.headline_projection = nn.Linear(self.hidden_size, self.reduced_size)
        self.article_projection = nn.Linear(self.hidden_size, self.reduced_size)
        
        self.cross_attention_1 = CrossAttention(self.reduced_size)
        self.cross_attention_2 = CrossAttention(self.reduced_size)
        
        self.classifier = nn.Sequential(
            nn.Linear(self.reduced_size * 4, self.reduced_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(self.reduced_size, self.reduced_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(self.reduced_size // 2, 2),
            nn.Softmax(dim=-1),
        )

    def forward(self, headline_ids, headline_mask, article_ids, article_mask):
        headline_output = self.bert(
            headline_ids,
            attention_mask=headline_mask,
            return_dict=True
        )
        
        article_output = self.bert(
            article_ids,
            attention_mask=article_mask,
            return_dict=True
        )
        
        headline_embeddings = headline_output.last_hidden_state
        article_embeddings = article_output.last_hidden_state
        
        headline_proj = self.headline_projection(headline_embeddings)
        article_proj = self.article_projection(article_embeddings)
        
        headline_attended = self.cross_attention_1(
            headline_proj, article_proj, article_proj, key_padding_mask=~article_mask.bool(),
        )
        article_attended = self.cross_attention_1(
            article_proj, headline_proj, headline_proj, key_padding_mask=~headline_mask.bool(),
        )
        
        headline_attended = self.cross_attention_2(
            headline_attended, article_attended, article_attended, key_padding_mask=~article_mask.bool(),
        )
        article_attended = self.cross_attention_2(
            article_attended, headline_attended, headline_attended, key_padding_mask=~headline_mask.bool(),
        )
        
        headline_pool = torch.mean(headline_attended, dim=1)
        article_pool = torch.mean(article_attended, dim=1)
        headline_max, _ = torch.max(headline_attended, dim=1)
        article_max, _ = torch.max(article_attended, dim=1)
        
        combined = torch.cat([headline_pool, article_pool, headline_max, article_max], dim=1)
        return self.classifier(combined)

In [11]:
def train_model(model, train_loader, val_loader, device, num_epochs=10):
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()
            
            inputs = {
                'headline_ids': batch['headline_ids'].to(device),
                'headline_mask': batch['headline_mask'].to(device),
                'article_ids': batch['article_ids'].to(device),
                'article_mask': batch['article_mask'].to(device)
            }
            labels = batch['labels'].to(device)
            
            try:
                outputs = model(**inputs)
                loss = criterion(outputs, labels)
                
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                
                if (batch_idx + 1) % 10 == 0:
                    print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {loss.item():.4f}')
                    
            except RuntimeError as e:
                print(f"Error in batch {batch_idx}: {str(e)}")
                print(f"Headline shape: {inputs['headline_ids'].shape}")
                print(f"Article shape: {inputs['article_ids'].shape}")
                continue
        
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = {
                    'headline_ids': batch['headline_ids'].to(device),
                    'headline_mask': batch['headline_mask'].to(device),
                    'article_ids': batch['article_ids'].to(device),
                    'article_mask': batch['article_mask'].to(device)
                }
                labels = batch['labels'].to(device)
                
                outputs = model(**inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
        
        print(f'Epoch {epoch + 1}:')
        print(f'Training Loss: {total_loss / len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss / len(val_loader):.4f}')

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

dataset = NewsConsistencyDataset(
    headlines=train_titles,
    article_bodies=train_contents,
    labels=labels,
    tokenizer=tokenizer,
    max_headline_len=128,
    max_article_len=1024,
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=35, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=35)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NewsConsistencyChecker().to(device)

train_model(model, train_loader, val_loader, device)

Error in batch 0: The expanded size of the tensor (1024) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [29, 1024].  Tensor sizes: [1, 512]
Headline shape: torch.Size([29, 128])
Article shape: torch.Size([29, 1024])


RuntimeError: The expanded size of the tensor (1024) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [8, 1024].  Tensor sizes: [1, 512]

In [ ]:
real_data = NewsConsistencyDataset(
    headlines=[""],
    article_bodies=[""],
    labels=[1],
    tokenizer=tokenizer,
    max_headline_len=128,
    max_article_len=1024,
)

data_loader = DataLoader(real_data, batch_size=1)

with torch.no_grad():
    for batch_idx, batch in enumerate(data_loader):
        inputs = {
            'headline_ids': batch['headline_ids'].to(device),
            'headline_mask': batch['headline_mask'].to(device),
            'article_ids': batch['article_ids'].to(device),
            'article_mask': batch['article_mask'].to(device)
        }

        outputs = model(**inputs)
        print(f"낚시성 기사일 확률: {round(outputs.numpy()[0][0]*100, 2)}%")

In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pt')
# loaded_model = NewsConsistencyChecker()
# loaded_model.load_state_dict(torch.load('model_state_dict.pt'))